In [1]:
#Laboratório 2 - UFABC - Laurindo dos Santos - 131710114

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

In [4]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print (type(palavrasRDD))

<class 'pyspark.rdd.RDD'>


In [5]:
# EXERCICIO
def Plural(palavra):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """

    return str.format(palavra + 's')

print (Plural('gato'))

gatos


In [6]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
    
    Returns:
        str: A string with 's' added to it.



In [7]:
assert Plural('rato')=='ratos', 'resultado incorreto!'
print ('OK')

OK


In [8]:
# EXERCICIO
pluralRDD = palavrasRDD.map(lambda x: Plural(x))
print (pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [9]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


In [10]:
pluralRDD = palavrasRDD.map(lambda x: str.format(x + 's'))
print (pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [11]:
# EXERCICIO
palavraPar = palavrasRDD.map(lambda x: (x, 1))
print (palavraPar.collect())

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


In [12]:
assert palavraPar.collect() == [('gato',1),('elefante',1),('rato',1),('rato',1),('gato',1)], 'valores incorretos!'
print ("OK")

OK


In [13]:
# EXERCICIO
palavrasGrupo = palavraPar.groupByKey()
for chave, valor in palavrasGrupo.collect():
    print ('{0}: {1}'.format(chave, list(valor)))

elefante: [1]
rato: [1, 1]
gato: [1, 1]


In [14]:
assert sorted(palavrasGrupo.mapValues(lambda x: list(x)).collect()) == [('elefante', [1]), ('gato',[1, 1]), ('rato',[1, 1])],'Valores incorretos!'
print ("OK")

OK


In [15]:
# EXERCICIO
contagemGroup = palavrasGrupo.mapValues(sum).sortByKey()
print (contagemGroup.collect())

[('elefante', 1), ('gato', 2), ('rato', 2)]


In [16]:
assert list(sorted(contagemGroup.collect()))==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [17]:
# EXERCICIO
contagem = palavraPar.reduceByKey(lambda x,y: x+y)
print (contagem.collect())

[('elefante', 1), ('rato', 2), ('gato', 2)]


In [18]:
assert sorted(contagem.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [19]:
# EXERCICIO
contagemFinal = (palavrasRDD
                     .map(lambda x: (x, 1))
                     .reduceByKey(lambda x,y: x+y)
                 )
print (contagemFinal.collect())

[('elefante', 1), ('rato', 2), ('gato', 2)]


In [20]:
assert sorted(contagemFinal.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [21]:
# EXERCICIO
palavrasUnicas = len(palavrasRDD.distinct().collect())
print (palavrasUnicas)

3


In [22]:
assert palavrasUnicas==3, 'valor incorreto!'
print ("OK")

OK


In [23]:
# EXERCICIO
# add é equivalente a lambda x,y: x+y
from operator import add
total = (contagemFinal
             .map(lambda x: (x[1]))
             .reduce(lambda x,y: x+y)
         )
media = total / float(palavrasUnicas)
print (total)
print (round(media, 2))

5
1.67


In [24]:
assert round(media, 2)==1.67, 'valores incorretos!'
print ("OK")

OK


In [25]:
# EXERCICIO
def contaPalavras(chavesRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        chavesRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (chavesRDD
                .map(lambda x: (x, 1))
                .reduceByKey(lambda x,y: x+y)
           )

print (contaPalavras(palavrasRDD).collect())

[('elefante', 1), ('rato', 2), ('gato', 2)]


In [26]:
assert sorted(contaPalavras(palavrasRDD).collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [27]:
# EXERCICIO
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
print (removerPontuacao('Ola, quem esta ai??!'))
print (removerPontuacao(' Sem espaco e_sublinhado!'))

ola quem esta ai
sem espaco esublinhado


In [28]:
assert removerPontuacao(' O uso de virgulas, embora permitido, nao deve contar. ')=='o uso de virgulas embora permitido nao deve contar', 'string incorreta!'
print ("OK")

OK


In [30]:
# Apenas execute a célula
import os.path
import urllib.request as urllib2

#url = 'http://www.gutenberg.org/cache/epub/100/pg100.txt' # url do livro

arquivo = 'C://Data//Aula02//shakespeare.txt' # local de destino: 'Data/Aula02/shakespeare.txt'

if os.path.isfile(arquivo):     # verifica se já fizemos download do arquivo
    print ('Arquivo já existe!')
else:
    try:        
        response = urllib2.urlopen(url)
        arquivo = (response.read()).split() #ja gera uma lista de palavras
    except IOError:
        print ('Impossível fazer o download: {0}'.format(url))

# lê o arquivo com textFile e aplica a função removerPontuacao        
shakesRDD = (sc.textFile(arquivo, 8).map(removerPontuacao))



Arquivo já existe!


In [31]:
print ('\n'.join(shakesRDD.zipWithIndex().map(lambda x: '{0}: {1}'.format(x[1], x[0])).take(15)))

0: the project gutenberg ebook of the complete works of william shakespeare by
1: william shakespeare
2: 
3: this ebook is for the use of anyone anywhere at no cost and with
4: almost no restrictions whatsoever  you may copy it give it away or
5: reuse it under the terms of the project gutenberg license included
6: with this ebook or online at wwwgutenbergorg
7: 
8: this is a copyrighted project gutenberg ebook details below
9: please follow the copyright guidelines in this file
10: 
11: title the complete works of william shakespeare
12: 
13: author william shakespeare
14: 


In [32]:
# EXERCICIO
shakesPalavrasRDD = shakesRDD.map(lambda x: x.split())
total = shakesPalavrasRDD.count()
print (shakesPalavrasRDD.take(5))
print (total)

[['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'by'], ['william', 'shakespeare'], [], ['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with'], ['almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or']]
124787


In [33]:
# EXERCICIO
shakesPalavrasRDD = shakesRDD.flatMap(lambda x: x.split())
total = shakesPalavrasRDD.count()
print (shakesPalavrasRDD.top(5))
print (total)

['zwaggerd', 'zounds', 'zounds', 'zounds', 'zounds']
903705


In [34]:
assert total==903705, 'valor incorreto!'
print ("OK")
assert shakesPalavrasRDD.top(5)==['zwaggerd', 'zounds', 'zounds', 'zounds', 'zounds'],'lista incorreta de palavras'
print ("OK")

OK
OK


In [35]:
# EXERCICIO
shakesLimpoRDD = shakesPalavrasRDD.filter(lambda x: x != '')
total = shakesLimpoRDD.count()
print (total)

903705


In [36]:
# EXERCICIO
top15 = contaPalavras(shakesLimpoRDD)
top15 = top15.takeOrdered(15, key = lambda x: -x[1])
print (top15)


[('the', 27825), ('and', 26791), ('i', 20681), ('to', 19261), ('of', 18289), ('a', 14667), ('you', 13716), ('my', 12481), ('that', 11135), ('in', 11027), ('is', 9621), ('not', 8745), ('for', 8261), ('with', 8046), ('me', 7769)]


In [37]:
assert top15 == [('the', 27825), ('and', 26791), ('i', 20681), ('to', 19261), ('of', 18289), ('a', 14667), ('you', 13716), 
                 ('my', 12481), ('that', 11135), ('in', 11027), ('is', 9621), ('not', 8745), ('for', 8261), ('with', 8046), 
                 ('me', 7769)],'valores incorretos!'
print ("OK")

OK


In [38]:
import numpy as np

# Vamos criar uma função pNorm que recebe como parâmetro p e retorna uma função que calcula a pNorma
def pNorm(p):
    """Generates a function to calculate the p-Norm between two points.

    Args:
        p (int): The integer p.

    Returns:
        Dist: A function that calculates the p-Norm.
    """

    def Dist(x,y):
        return np.power(np.power(np.abs(x-y),p).sum(),1/float(p))
    return Dist

In [39]:
# Vamos criar uma RDD com valores numéricos
numPointsRDD = sc.parallelize(enumerate(np.random.random(size=(10,100))))

In [40]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartPointsRDD = numPointsRDD.cartesian(numPointsRDD)

In [42]:
cartPoints0RDD = cartPointsRDD.map(lambda x: x[0])

In [43]:
cartPoints1RDD = cartPointsRDD.map(lambda x: x[1])

In [ ]:

# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartPointsRDD = numPointsRDD.<COMPLETAR>

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartPointsParesRDD = cartPointsRDD.<COMPLETAR>


# Aplique um mapa para calcular a Distância Euclidiana entre os pares
Euclid = pNorm(2)
distRDD = cartPointsParesRDD.<COMPLETAR>

# Encontre a distância máxima, mínima e média, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o cálculo da min, max, mean
statRDD = distRDD.<COMPLETAR>

minv, maxv, meanv = statRDD.<COMPLETAR>, statRDD.<COMPLETAR>, statRDD.<COMPLETAR>
print minv, maxv, meanv


In [ ]:
assert (minv.round(2), maxv.round(2), meanv.round(2))==(0.0, 4.70, 3.65), 'Valores incorretos'
print "OK"

In [114]:
cartPoints1RDD.map(lambda x: x[1])

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [312]:
cartPointsRDD.map(lambda x: x[0].split(",")).take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 199.0 failed 1 times, most recent failure: Lost task 0.0 in stage 199.0 (TID 809, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\Cristiano Passos\Anaconda3\lib\pyspark\rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-312-b143d7ee3b88>", line 1, in <lambda>
AttributeError: 'tuple' object has no attribute 'split'



In [44]:
# Vamos criar uma função para calcular a distância de Hamming
def Hamming(x,y):
    """Calculates the Hamming distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        H (int): The Hamming distance between x and y.
    """
    return (x!=y).sum()

# Vamos criar uma função para calcular a distância de Jaccard
def Jaccard(x,y):
    """Calculates the Jaccard distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        J (int): The Jaccard distance between x and y.
    """
    return (x==y).sum()/float( np.maximum(x,y).sum() )

In [63]:
# Vamos criar uma RDD com valores categóricos
catPointsRDD = sc.parallelize(enumerate([['alto', 'caro', 'azul'],
                                         ['medio', 'caro', 'verde'],
                                         ['alto', 'barato', 'azul'],
                                         ['medio', 'caro', 'vermelho'],
                                         ['baixo', 'barato', 'verde'],
                                        ]))

In [83]:
# EXERCICIO
# Crie um RDD de chaves únicas utilizando flatMap
chavesRDD = (catPointsRDD
                .flatMap(lambda x: x[1])
                .distinct()
                .collect()
             )

#catPointsRDD.flatMap(lambda x: x[1]).distinct().collect()

chaves = dict((v,k) for k,v in enumerate(chavesRDD))
nchaves = len(chaves)
print (chaves, nchaves)

{'baixo': 0, 'barato': 1, 'alto': 2, 'medio': 3, 'verde': 4, 'vermelho': 5, 'caro': 6, 'azul': 7} 8


In [84]:

assert chaves=={'baixo': 0, 'barato': 1, 'alto': 2, 'medio': 3, 'verde': 4, 'vermelho': 5, 'caro': 6, 'azul': 7}, 'valores incorretos!'
print ("OK")

assert nchaves==8, 'número de chaves incorreta'
print ("OK")


OK
OK


In [85]:
def CreateNP(atributos,chaves):  
    """Binarize the categorical vector using a dictionary of keys.

    Args:
        atributos (list): List of attributes of a given object.
        chaves (dict): dictionary with the relation attribute -> index

    Returns:
        array (np.array): Binary array of attributes.
    """
    
    array = np.zeros(len(chaves))
    for atr in atributos:
        array[ chaves[atr] ] = 1
    return array

# Converte o RDD para o formato binário, utilizando o dict chaves
binRDD = catPointsRDD.map(lambda rec: (rec[0],CreateNP(rec[1], chaves)))
binRDD.collect()

[(0, array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.])),
 (1, array([ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.])),
 (2, array([ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.])),
 (3, array([ 0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.])),
 (4, array([ 1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.]))]

In [86]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartBinRDD = binRDD.cartesian(binRDD)

In [111]:
cartBinRDD.map(lambda x: x).flatMap(lambda x: x).take(8)

[(0, array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.])),
 (0, array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.])),
 (0, array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.])),
 (1, array([ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.])),
 (0, array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.])),
 (2, array([ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.])),
 (0, array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.])),
 (3, array([ 0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.]))]

In [88]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartBinRDD = binRDD.<COMPLETAR>

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartBinParesRDD = cartBinRDD.<COMPLETAR>


# Aplique um mapa para calcular a Distância de Hamming e Jaccard entre os pares
hamRDD = cartBinParesRDD.<COMPLETAR>
jacRDD = cartBinParesRDD.<COMPLETAR>

# Encontre a distância máxima, mínima e média, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o cálculo da min, max, mean
statHRDD = hamRDD.<COMPLETAR>
statJRDD = jacRDD.<COMPLETAR>

Hmin, Hmax, Hmean = statHRDD.<COMPLETAR>, statHRDD.<COMPLETAR>, statHRDD.<COMPLETAR>
Jmin, Jmax, Jmean = statJRDD.<COMPLETAR>, statJRDD.<COMPLETAR>, statJRDD.<COMPLETAR>

print "\t\tMin\tMax\tMean"
print "Hamming:\t{:.2f}\t{:.2f}\t{:.2f}".format(Hmin, Hmax, Hmean )
print "Jaccard:\t{:.2f}\t{:.2f}\t{:.2f}".format( Jmin, Jmax, Jmean )

TypeError: 'RDD' object is not iterable

In [ ]:
assert (Hmin.round(2), Hmax.round(2), Hmean.round(2)) == (0.00,6.00,3.52), 'valores incorretos'
print "OK"
assert (Jmin.round(2), Jmax.round(2), Jmean.round(2)) == (0.33,2.67,1.14), 'valores incorretos'
print "OK"